TP Siluroformes de Benjamin Forestier

In [ ]:
import pandas
import matplotlib.pyplot as plt

siluriforme = pandas.read_csv('catfish.csv', index_col='Date', parse_dates=True)
siluriforme.head(1)
siluriforme.tail(-1)

plt.rcParams["figure.figsize"] = (10, 5)
siluriforme.plot()

In [ ]:
subset = siluriforme.loc['1996-1-01':'2000-1-01'] # Exemple : de janvier à juin 2023
subset.plot()
plt.show()


Moyennes des ventes :

In [ ]:
import numpy as np

# Année 92
year = siluriforme.loc['1992']
print(f"1992 : {np.mean(year)}")

# Décennie 90
decade = siluriforme.loc['1990-1-01':'1999-12-31']
print(f"90 : {np.mean(decade)}")

# Ensemble du dataset
print(f"Dataset : {np.mean(siluriforme)}")

Médianne des ventes :

In [ ]:
# Année 92
year = siluriforme.loc['1992']
print(f"1992 : {np.median(year)}")

# Décennie 90
decade = siluriforme.loc['1990-1-01':'1999-12-31']
print(f"90 : {np.median(decade)}")

# Ensemble du dataset
print(f"Dataset : {np.median(siluriforme)}")

Mode des ventes :

In [ ]:
# Année 1992
year = siluriforme.loc['1992']
value_counts_year = year["Total"].value_counts()
modes = year["Total"].mode()
print("Année 1992 :")
for mode in modes:
    print(f"{mode} apparaît {value_counts_year[mode]} fois")
# print(year["Total"].mode)

# Décennie 90
decade = siluriforme.loc['1990-1-01':'1999-12-31']
value_counts_decade = decade["Total"].value_counts()
modes = decade["Total"].mode()
print("Décennie 90 :")
for mode in modes:
    print(f"{mode} apparaît {value_counts_decade[mode]} fois")
# print(decade["Total"].mode)

# Ensemble du dataset
value_counts_dataset = siluriforme["Total"].value_counts()
modes = siluriforme["Total"].mode()
print("Ensemble du dataset :")
for mode in modes:
    print(f"{mode} apparaît {value_counts_dataset[mode]} fois")
# print(siluriforme["Total"].mode)

On peut en conclure qu'il n'y a jamais exactement la même quantité de siluriformes bien que les valeurs tournent autour des 15000, 18000 et 19000.

Ecart-type des ventes :

In [ ]:
# Année 1992
year = siluriforme.loc['1992']
print(f"L'écart-type pour l'année 1992 est : {year.std()}")

# Décennie 90
decade = siluriforme.loc['1990-1-01':'1999-12-31']
print(f"L'écart-type pour la décennie 90 est : {decade.std()}")

# Ensemble du dataset
print(f"L'écart-type pour l'ensemble du dataset est : {siluriforme.std()}")

Série stationnaire ?

In [ ]:
rolling_mean = siluriforme.rolling(window=12).mean()
rolling_std = siluriforme.rolling(window=12).std()

plt.plot(siluriforme, color='b', label='Origine')
plt.plot(rolling_mean, color='r', label='Moyenne mobile')
plt.plot(rolling_std, color='g', label='Ecart-type mobile')
plt.legend(loc='best')
plt.title('Moyenne et Ecart-type mobiles')
plt.show()

On pouvait déjà s'en rendre compte avec le graphique de la question 2 représentant la courbe des ventes. Mais les statistiques roulantes augmentant et diminuant avec le temps nous prouve bel et bien que cette série temporelle n'est pas stationnaire.

Saisonnalité ?

In [ ]:
log_siluriforme = np.log(siluriforme)
plt.figure(figsize=(12, 6))

years = [str(year) for year in log_siluriforme.index.year.unique()]
months = log_siluriforme.index.month_name().unique()
colors = plt.cm.inferno(np.linspace(0, 1, len(years)))

for index, year in enumerate(years):
    plt.plot(months, log_siluriforme.loc[year], label=year, color = colors[index])
    plt.legend(bbox_to_anchor=(1, 1))

On peut en effet voir ce qui ressemble à une saisonnalité annuelle, le grapahique ci-dessus le montre, bien qu'a partir des années 2006 - 2007 les courbes se déforme un peu mais puisque les ventes s'effondrent mais en gardant une certaine forme de saisonnalité.

Tendance ?

In [ ]:
windows = list(range(3, 25, 3)) + [36]
fig = plt.figure(figsize=(12, 8))
for index, window in enumerate(windows, start=1):
    ax = fig.add_subplot(3, int(len(windows) / 3), index)
    ax.plot(log_siluriforme)
    ax.plot(log_siluriforme.rolling(window, center=True).mean(), label=f'Moyenne sur {window} mois')
    ax.legend()

On constate une tendance croissante entre 1988 et 2004 et décroissante entre 2004 et 2012. Cette tendance suggère en effet une notionde saisonnalité.

Modéle prédictif à partir des donnéees de 1986 à 2000

In [ ]:
from statsmodels.tsa.api import graphics
from statsmodels.tsa.stattools import adfuller

def get_stationarity(timeseries, window=12):
    # Statistiques mobiles
    rolling_mean = timeseries.rolling(window=window).mean()
    rolling_std = timeseries.rolling(window=window).std()

    # tracé statistiques mobiles
    original = plt.plot(timeseries, color='b', label='Origine')
    mean = plt.plot(rolling_mean, color='r', label='Moyenne Mobile')
    std = plt.plot(rolling_std, color='g', label='Ecart-type Mobile')
    plt.legend(loc='best')
    plt.title('Moyenne et écart-type Mobiles')
    plt.show(block=False)

    # Test Dickey–Fuller :
    result = adfuller(timeseries)
    print('Statistiques ADF : {}'.format(result[0]))
    print('p-value : {}'.format(result[1]))
    print('Valeurs Critiques :')
    for key, value in result[4].items():
        print('\t{}: {}'.format(key, value))

    graphics.plot_acf(timeseries, lags=None)

In [ ]:
rolling_mean = log_siluriforme.rolling(window=12).mean()
df_log_minus_mean = log_siluriforme - rolling_mean
df_log_minus_mean.dropna(inplace=True)

get_stationarity(df_log_minus_mean, 12)

In [ ]:
rolling_mean_exp_decay = log_siluriforme.ewm(halflife=12, min_periods=0, adjust=True).mean()
df_log_exp_decay = log_siluriforme - rolling_mean_exp_decay
df_log_exp_decay.dropna(inplace=True)
get_stationarity(df_log_exp_decay, 12)

In [ ]:
df_log_shift = log_siluriforme - log_siluriforme.shift()
df_log_shift.dropna(inplace=True)
get_stationarity(df_log_shift, 12)

In [ ]:
differenced = siluriforme.diff().dropna()
get_stationarity(differenced, 12)

In [ ]:
from statsmodels.tsa.api import graphics

def plot_acf_pacf(timeseries):
    fig = plt.figure(figsize=(12, 18))
    for index, (timeserie_title, timeserie) in enumerate(timeseries.items()):
        index = index * 2
        ax = fig.add_subplot(len(timeseries), 2, index + 1)
        ax.title.set_text(timeserie_title)
        graphics.plot_acf(timeserie, ax=ax)
        ax.title.set_text('ACF %s' % timeserie_title)

        ax = fig.add_subplot(len(timeseries), 2, index + 2)
        graphics.plot_pacf(timeserie, ax=ax)
        ax.title.set_text('PACF %s' % timeserie_title)

plot_acf_pacf({
    'Soustraction de la moyenne': df_log_minus_mean,
    'Décroissance exponentielle': df_log_exp_decay,
    'Décalage temporel': df_log_shift,
    'DIFF': differenced

})

p = 2
d = 1
q = 1
ARIMA(2,1,1)

In [ ]:
train_data = siluriforme.loc['1986':'2000']
test_data = siluriforme.loc['2001':'2012']

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Créer un modèle ARIMA(2,1,1)
model = ARIMA(train_data, order=(2,1,1), freq='MS')

# Ajuster le modèle
model_fit = model.fit()

# Afficher le résumé du modèle
print(model_fit.summary())

In [ ]:
residuals = model_fit.resid
plot_acf_pacf({
    'Residue modèle': residuals
})

In [ ]:
train_predictions = model_fit.predict(start=train_data.index[0], end=train_data.index[-1])

# Prédictions sur l'ensemble de test
test_predictions = model_fit.predict(start=test_data.index[0], end=test_data.index[-1])

# Prédictions sur l'ensemble d'entraînement
train_predictions = model_fit.predict(start=train_data.index[0], end=train_data.index[-1])

# Prédictions sur l'ensemble de test
test_predictions = model_fit.predict(start=test_data.index[0], end=test_data.index[-1])

# Tracer les prédictions et les données réelles avec les courbes collées
plt.figure(figsize=(10, 6))
plt.plot(train_data.index, train_data.values, label='Ensemble d\'entraînement', color='blue')
plt.plot(test_data.index, test_data.values, label='Ensemble de test (réel)', color='blue', linestyle='--')
plt.plot(train_predictions.index, train_predictions, label='Prédictions (Entraînement)', color='red')
plt.plot(test_predictions.index, test_predictions, label='Prédictions (Test)', color='green')

# Définir les limites des axes x et y pour une continuité visuelle
#plt.xlim(train_data.index[0], test_data.index[-1])
#plt.ylim(min(train_data.min(), test_data.min()), max(train_data.max(), test_data.max()))

plt.xlabel('Date')
plt.ylabel('Siluriformes vendus')
plt.title('Prédictions du modèle ARIMA')
plt.legend()
plt.show()

Selon le modèle que j'ai trouvé les prédictions de test sont complétement ratés, j'obtiens une ligne droite. Les modèles peuvent ne pas être justes, ils faut donc les prendre avec des pincettes

Modéle prédictif avec la librairie pmdarima

In [ ]:
import pmdarima as pm

# Séparer les données en ensemble d'entraînement et ensemble de test
train_data = siluriforme.loc['1986':'2000']
test_data = siluriforme.loc['2001':'2012']

# Utiliser auto_arima pour trouver le meilleur modèle ARIMA
model = pm.auto_arima(train_data)

print(model.summary())

In [ ]:
# Ajuster le modèle aux données
model.fit(train_data)
# Obtenir les résidus du modèle
residuals = model.resid()
plot_acf_pacf({
    'Residue modèle': residuals
})

In [ ]:
# Faire des prédictions sur l'ensemble d'entraînement
train_pred, train_confint = model.predict_in_sample(return_conf_int=True)

# Faire des prédictions sur l'ensemble de test
n_periods = len(test_data)
predicted, confint = model.predict(n_periods=n_periods, return_conf_int=True)

# Concaténer les prédictions pour l'ensemble d'entraînement et de test
all_predictions = pandas.concat([pandas.Series(train_pred, index=train_data.index),
                            pandas.Series(predicted, index=test_data.index)],
                            axis=0)

# Tracer les valeurs réelles et les prédictions pour l'ensemble d'entraînement et de test
plt.figure(figsize=(12, 6))
plt.plot(train_data, label='Observed Training', color='blue')
plt.plot(test_data, label='Observed Test', color='green')
plt.plot(all_predictions, label='Predicted', color='red')

plt.xlabel('Date')
plt.ylabel('Siluriformes vendus')
plt.title('Observed vs Predicted Passengers')
plt.legend()
plt.grid(True)
plt.show()

Le modèle généré avec la librairie garde la saisonalité cependant, ont observe une stationnarisation des données crée alors que les données observé indique clairement une décroissance du nombre de silriformes vendus.

Poisson-chat et loi normale

Entre 10 et 25 cm ?
Moins de 25 cm ?
Plus de 30 cm ?

In [ ]:
from statistics import NormalDist

mu = 25
sigma = 7.5

dist = NormalDist(mu, sigma)

p_between_10_25 = dist.cdf(25) - dist.cdf(10)  # P(10 ≤ X ≤ 25)
p_less_25 = dist.cdf(25)               # P(X ≤ 25)
p_more_30 = 1 - dist.cdf(30)           # P(X ≥ 30)

p_between_10_25 = round(p_between_10_25, 3)
p_less_25 = round(p_less_25, 3)
p_more_30 = round(p_more_30, 3)

print(f"La proba qu'un poisson est une taille entre 10 et 25 cm est : P(10 ≤ X ≤ 25) = {p_between_10_25}")
print(f"La proba qu'un poisson est une taille de moins de 25 cm est : P(X ≤ 25) = {p_less_25}")
print(f"La proba qu'un poisson est une taille de plus 30 cm est : P(X ≥ 30) = {p_more_30}")
